<a href="https://colab.research.google.com/github/thePhukan/Summarize/blob/parent/mistral_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=81a031545260774b3cdf852aa5f09a25c69d914c99406aa4ca72d47c68c77c4a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 24.2 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "mistralai/Mistral-7B-v0.3"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(model_id)
# inputs = tokenizer("Hello my name is", return_tensors="pt")

# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
from transformers import AutoModelForCausalLM, AutoTokenizer
import os


# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # For CPU
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'


# -*- coding: utf-8 -*-
"""
Created on Tue Aug 20 18:39:58 2024

@author: arpan
"""
import transformers
import pickle
# with open("D:/Anas/document_database.pkl", "rb") as f:
#     pkl = pickle.load(f)
import pandas as pd
from rouge_score import rouge_scorer
# path = "/DATA/priyanshu_2021cs26/arpan/webqa/"
path = "/content/drive/MyDrive/Webqa/rag/"
# from datasets import load_metric,Dataset,DatasetDict
from transformers import LlamaForCausalLM, DataCollatorForSeq2Seq,TrainingArguments,AutoModelForCausalLM,AutoTokenizer,LlamaTokenizer,Seq2SeqTrainingArguments,Seq2SeqTrainer
from trl import SFTTrainer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# os.environ['CUDA_VISIBLE_DEVICES'] = "3"
os.environ['WANDB_DISABLED']="True"
import torch
torch.cuda.empty_cache()
import torch.nn as nn

import json, random, time, os, base64
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
np.set_printoptions(precision=4)
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from io import BytesIO

is_cuda = torch.cuda.is_available()
device = torch.device("cuda" if is_cuda else "cpu")
from peft import PeftModel, PeftConfig



import os
import re

# Define the regular expression for matching URLs
url_pattern = re.compile(r'https?://\S+')

print("\nDEVICE:\t",device)

# model_checkpoint = "meta-llama/Meta-Llama-3-8B"
# model_checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# peftconfig = PeftConfig.from_pretrained(peft_model_id)
peft_config = LoraConfig(
    task_type="CAUSAL_LM", inference_mode=True, r=2, lora_alpha=2, lora_dropout=0.3,
target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","gate_proj",
                "down_proj"]
)
max_input_length = 3000 #2600 #4600    #256
max_target_length = 3000
# model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto")
# model = get_peft_model(model, peft_config)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3").to(device)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

# from torch.nn.parallel import DataParallel
# model = nn.DataParallel(model)
# model = get_peft_model(model.module, peft_config)
model = get_peft_model(model, peft_config)
# model=model.module
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
model.eval()

with open(path+"document_database_TEST.txt", "r", encoding="utf-8") as f3:
        test1 = f3.read()


def clean_text(text):
    # Remove multiple newlines or spaces
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)

    # Remove typical Wikipedia header/footer text patterns
    text = re.sub(r'(Jump to navigation|Jump to search|Contents|See also|References|Further reading|External links|Wikipedia|to main Search sidebar)', '', text, flags=re.IGNORECASE)

    # Remove sections that are likely menus or sidebars
    text = re.sub(r'(edit|view history|related changes|special pages|permanent link|page information|Cite this page|download as PDF|Login|Sign up|sidebar)', '', text, flags=re.IGNORECASE)

    # Remove repeated "citation needed" or similar patterns
    text = re.sub(r'\[citation needed\]', '', text, flags=re.IGNORECASE)

    # Remove extra symbols like '[edit]', '[source]', etc.
    text = re.sub(r'\[.*?\]', '', text)

    # Remove remaining irrelevant content based on patterns you find
    # Example: remove lines that start with "Retrieved from"
    text = re.sub(r'^Retrieved from.*$', '', text, flags=re.MULTILINE)

    return text.strip()

test1 = clean_text(test1)
test1 = re.sub(url_pattern, '', test1)
# splits = text_splitter.split_documents(test1)
placeholder = "__PARA_SEPARATOR__"
text = (test1).replace("\n\n", placeholder)

text = (text).replace('\n', '')
text = re.sub(url_pattern, '', text)
test1 = text.replace(placeholder, "\n\n")

# text = [str("You are given the text from a wikipedia webpage. Summarize the content for the whole webpage within 524 words. Web Page Content: ")+test1]
# text = [str("Summarize: Web Page Content: ")+test1]
# text = [str("Summarize the main points and key details from the following Wikipedia article: ")+test1+"Please provide a concise summary that captures the essential information while excluding unnecessary details. The summary should be coherent, informative, and suitable for a general audience."]


# text = [str("Please summarize the key points and main details from the following Wikipedia article text. Focus on the essential facts, structure, and primary information while avoiding any irrelevant content, markup, or links.\n")
# +test1+
# "Provide a concise and coherent summary that captures the article's main ideas."]

# text = [str("Please summarize the key points and main details from the following Wikipedia article text in 3-5 sentences. Focus on the essential facts, structure, and primary information while avoiding any irrelevant content, markup, or links.\n")
# +test1+
# "Provide a concise and coherent summary that captures the article's main ideas."]


# text = [str("Please summarize the key points and main details in 3-5 sentences. Focus on the essential facts, structure, and primary information while avoiding any irrelevant content, markup, or links.\n")
# +test1+
# "Provide a concise and coherent summary that captures the article's main ideas."]

# print("test1:",test1)
# text = [str("Please summarize in 3-5 sentences.\n")
# +test1]

messages = [
    {"role": "system", "content": "Please summarize the key points and main details in 200 words"},
    {"role": "user", "content": test1},
]



# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",  max_length=3000, truncation=True).to("cuda")
# model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",  max_length=3000, truncation=True)

generated_ids = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
print(tokenizer.batch_decode(generated_ids)[0])


DEVICE:	 cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]